In [1]:
!pip install -f https://download.pytorch.org/whl/test/cpu/torch_test.html torch==1.8.0 torchtext==0.9.0 torchvision==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/test/cpu/torch_test.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.0
    Uninstalling torchtext-0.14.0:
      Successfully uninstalled torchtext-0.14.0
ERROR: pip's dependency resolver does not currently tak

In [3]:
!python -m spacy download en

2023-01-13 06:13:19.177045: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import torch
from torchtext.legacy import data

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en')
LABEL = data.LabelField(dtype = torch.float) # pos -> 1 / neg -> 0

In [7]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 52.4MB/s]


In [8]:
print(f'training examples 수 : {len(train_data)}')
print(f'testing examples 수 : {len(test_data)}')

training examples 수 : 25000
testing examples 수 : 25000


In [9]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, min_freq = 5)
LABEL.build_vocab(train_data)

In [25]:
train_data, valid_data = train_data.split(split_ratio=0.8)

In [44]:
print(f'training examples 수 : {len(train_data)}')
print(f'validations examples 수 : {len(valid_data)}')
print(f'testing examples 수 : {len(test_data)}')

training examples 수 : 12800
validations examples 수 : 3200
testing examples 수 : 25000


In [27]:
BATCH_SIZE = 128

train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
)

In [113]:
import torch
import torch.utils.data as Data
from torch import nn
from torch.nn import functional as F
from torch import optim


In [127]:
class Model(nn.Module):
    def __init__(self, n_feature):
        super().__init__()
        self.embedding = nn.Embedding(n_feature, 64)
        self.rnn = nn.RNN(64, 32)
        self.Classifier = nn.Linear(32, 1)
    
    def forward(self, x):
        x = self.embedding(x)
        x, hidden = self.rnn(x)
        return self.Classifier(hidden[-1,:,:])

In [128]:
n_feature = len(TEXT.vocab)

In [129]:
model = Model(n_feature)

In [130]:
criterion = nn.BCEWithLogitsLoss() # softmax가 내장

In [131]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [132]:
import numpy as np

In [133]:
def binary_accuracy(preds, y):

  rounded_preds = torch.round(torch.sigmoid(preds))
  # rounded_preds : [batch size]
  # y : batch.label
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

In [135]:
epoch_n = 10

for epoch in range(epoch_n):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data.text).squeeze(1)
        loss = criterion(output, data.label)

        acc = binary_accuracy(output, data.label)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        loss_epoch, acc =epoch_loss / len(train_loader), epoch_acc / len(train_loader)

    model.eval()
    epoch_v_loss = 0
    epoch_v_acc = 0
    for data in valid_loader:
        output = model(data.text).squeeze(1)
        loss = criterion(output, data.label)
        acc = binary_accuracy(output, data.label)
    
        epoch_v_loss += loss.item()
        epoch_v_acc += acc.item()
        loss_epoch_v, acc_v =epoch_loss / len(train_loader), epoch_acc / len(train_loader)

    print(f'Epoch {epoch+1}/{epoch_n}: train_Acc: {acc:.4f}, train_Loss{loss_epoch:.8f}, valid_Acc: {acc_v:.4f}, valid_Loss{loss_epoch_v:.8f}')


Epoch 1/10: train_Acc: 0.4531, train_Loss0.69403444, valid_Acc: 0.4981, valid_Loss0.69403444
Epoch 2/10: train_Acc: 0.5391, train_Loss0.69373156, valid_Acc: 0.4998, valid_Loss0.69373156
Epoch 3/10: train_Acc: 0.4531, train_Loss0.69339442, valid_Acc: 0.4963, valid_Loss0.69339442
Epoch 4/10: train_Acc: 0.4531, train_Loss0.69304368, valid_Acc: 0.5052, valid_Loss0.69304368
Epoch 5/10: train_Acc: 0.5391, train_Loss0.69281675, valid_Acc: 0.4992, valid_Loss0.69281675
Epoch 6/10: train_Acc: 0.4531, train_Loss0.69235480, valid_Acc: 0.5045, valid_Loss0.69235480
Epoch 7/10: train_Acc: 0.5391, train_Loss0.69233362, valid_Acc: 0.5051, valid_Loss0.69233362
Epoch 8/10: train_Acc: 0.4531, train_Loss0.69184903, valid_Acc: 0.5072, valid_Loss0.69184903
Epoch 9/10: train_Acc: 0.4531, train_Loss0.69242696, valid_Acc: 0.4958, valid_Loss0.69242696
Epoch 10/10: train_Acc: 0.4531, train_Loss0.69161433, valid_Acc: 0.4999, valid_Loss0.69161433


In [137]:
model.eval()
epoch_v_loss = 0
epoch_v_acc = 0
for data in test_loader:
    output = model(data.text).squeeze(1)
    loss = criterion(output, data.label)
    acc = binary_accuracy(output, data.label)

    epoch_v_loss += loss.item()
    epoch_v_acc += acc.item()
    loss_epoch_v, acc_v =epoch_loss / len(train_loader), epoch_acc / len(train_loader)

print(f'Epoch {epoch+1}/{epoch_n}: Test_Acc: {acc_v:.4f}, Test_Loss{loss_epoch_v:.8f}')


Epoch 10/10: Test_Acc: 0.4999, Test_Loss0.69161433


In [112]:
# 1-layer 결과
epoch_n = 10

for epoch in range(epoch_n):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for data in train_loader:
        optimizer.zero_grad()

        output = model(data.text).squeeze(1)
        loss = criterion(output, data.label)

        acc = binary_accuracy(output, data.label)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        loss_epoch, acc =epoch_loss / len(train_loader), epoch_acc / len(train_loader)

    model.eval()
    epoch_v_loss = 0
    epoch_v_acc = 0
    for data in valid_loader:
        output = model(data.text).squeeze(1)
        loss = criterion(output, data.label)
        acc = binary_accuracy(output, data.label)
    
        epoch_v_loss += loss.item()
        epoch_v_acc += acc.item()
        loss_epoch_v, acc_v =epoch_loss / len(train_loader), epoch_acc / len(train_loader)

    print(f'Epoch {epoch+1}/{epoch_n}: train_Acc: {acc:.4f}, train_Loss{loss_epoch:.8f}, valid_Acc: {acc_v:.4f}, valid_Loss{loss_epoch_v:.8f}')


Epoch 1/10: train_Acc: 0.5469, train_Loss0.69682066, valid_Acc: 0.4944, valid_Loss0.69682066
Epoch 2/10: train_Acc: 0.4609, train_Loss0.69390388, valid_Acc: 0.4988, valid_Loss0.69390388
Epoch 3/10: train_Acc: 0.5391, train_Loss0.69364769, valid_Acc: 0.5034, valid_Loss0.69364769
Epoch 4/10: train_Acc: 0.5391, train_Loss0.69347102, valid_Acc: 0.4974, valid_Loss0.69347102
Epoch 5/10: train_Acc: 0.5391, train_Loss0.69317587, valid_Acc: 0.4976, valid_Loss0.69317587
Epoch 6/10: train_Acc: 0.4531, train_Loss0.69297602, valid_Acc: 0.5038, valid_Loss0.69297602
Epoch 7/10: train_Acc: 0.4531, train_Loss0.69291159, valid_Acc: 0.5074, valid_Loss0.69291159
Epoch 8/10: train_Acc: 0.4531, train_Loss0.69298158, valid_Acc: 0.4985, valid_Loss0.69298158
Epoch 9/10: train_Acc: 0.4531, train_Loss0.69254221, valid_Acc: 0.4996, valid_Loss0.69254221
Epoch 10/10: train_Acc: 0.4531, train_Loss0.69225293, valid_Acc: 0.5021, valid_Loss0.69225293
